In [ ]:

@app_home_page.route('/our_recommendations_list', methods=['GET'])
def our_recommendations_list():
    result ={}
    result["recommendations_enable"] ={}
    result["recommendations_disable"] ={}
    try:
        if not session.get("adminId"):
            return redirect(url_for('admin.login'))
        data_status = {"responseStatus": 0, "result": ""}
        result["recommendations_enable"] = json.loads(PiesaGivingOrganisations.objects.all().filter(recommendation_status=1).to_json())
        result["recommendations_disable"] = json.loads(PiesaGivingOrganisations.objects.all().filter(recommendation_status=0).to_json())
        data_status["responseStatus"] = 1
        data_status["result"] = "Our recommendations fetched successfully!"
    except Exception as err:
        app.logger.error(traceback.format_exc())
        pass
    return render_template('/super_admin/our_recommendations.html', result = result)

@app_home_page.route('/modify_our_recomendations', methods=['POST'])
def modify_our_recomendations():
    result = {}
    try:
        data_status = {"responseStatus": 0, "result": ""}
        if not session.get("adminId"):
            return redirect(url_for('admin.login'))
        query = PiesaGivingOrganisations.objects(recommendation_status=1)
        query.update(recommendation_status=0)
        for i in list(request.form):
            _id = i.split(":")[-1].strip("}").strip('"').strip("'").strip().strip("'")
            queryset = PiesaGivingOrganisations.objects.get(id__iexact=_id)
            if queryset:
                queryset.recommendation_status = 1
                queryset.save()
        data_status["responseStatus"] = 1
        data_status["result"] = "Our recommendations modified successfully!"
        flash("Our recommendations modified successfully!")
    except Exception as err:
        app.logger.error(traceback.format_exc())
        pass
    return redirect(url_for('app_home_page.our_recommendations_list'))


In [ ]:
                    if admin_created:
                        if profilePic.filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS:
                            ext = profilePic.filename.rsplit('.',1)[1].lower()
                            file_name = str(admin_created.id)+"."+ext
                            if not os.path.exists(app.config['UPLOAD_FOLDER']+"/admin"):
                                os.mkdir(app.config['UPLOAD_FOLDER']+"/admin")
                            admin_image = app.config['UPLOAD_FOLDER']+"/admin"
                            profilePic.save(os.path.join(admin_image,file_name))
                        admin_created.update(profilePic=file_name)
                        return redirect(url_for('admin.login'))

In [ ]:
	
# Image Folder Settings for Admin and Department
SITE_ROOT = os.path.dirname(os.path.realpath(__file__))
UPLOAD_FOLDER = SITE_ROOT + '/media/images'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# UPLOAD_PDF = SITE_ROOT + '/media/upload_purchase'
# app.config['UPLOAD_FOLDER'] = UPLOAD_PDF
app.config['SITE_ROOT']=SITE_ROOT


In [ ]:
#c email Regex


                    if not (re.search(email_regex, email)):
                        error = "email is not valid!!!"

In [ ]:
#Church List And Edit 
@admin_organizations.route('/church_list', methods=['GET','POST'])
def church_list():
    result = []
    try:
        data_status = {"responseStatus": 0, "result": ""}
        if not session.get("adminId"):
            return redirect(url_for('admin.login'))

        if request.method == "GET":
            church_data = PiesaGivingOrganisations.objects(status__in=[0,1], organizationType = 'church',categoryId=ObjectId("5ef2c94747b3161c07ced218")).order_by('-id').all()
        if request.method == "POST":
            church_data = []
            searchElement = request.form.get("searchElement","")
            queryset_name = PiesaGivingOrganisations.objects.filter(status__in=[0,1], organizationType = 'church', displayName__icontains=str(searchElement)).all()
            queryset_email = PiesaGivingOrganisations.objects.filter(status__in=[0,1], organizationType = 'church', email__icontains=str(searchElement)).all()
            queryset_project = PiesaGivingOrganisations.objects.filter(status__in=[0,1], organizationType = 'church', categoryId__in=PiesaGivingCategories.objects(categoryName__icontains=str(searchElement))).all()
            church_data.extend(queryset_name)
            church_data.extend(queryset_email)
            church_data.extend(queryset_project)
        
        if church_data:
            for church in church_data:
                church_dict = {
                    'id':str(church.id),
                    'displayName':church.displayName,
                    # 'categoryName':str(church.categoryId.categoryName),
                    'email':church.email,
                    "profilePic":church.profilePic,
                    "organizationType":church.organizationType
                }
                if church.status == 1:
                    church_dict["status"] = "Active"
                elif church.status == 0:
                    church_dict["status"] = "Inactive"
                if church_dict not in result:
                    result.append(church_dict)
            data_status["responseStatus"] = 1
            data_status["result"] = "Church list fetched successfully!"
            data_status["data"] = result

        else:
            data_status["responseStatus"] = 0
            data_status["result"] = "Data Not Available"
    except Exception as err:
        app.logger.error(traceback.format_exc())
        pass
    return render_template('/super_admin/church_list.html', result = result) 


# edit church functionality
@admin_organizations.route('/edit_organizations', methods=['POST'])
def edit_organizations():
    id = request.form['editid']
    organizationType = request.form['organizationType']
    try:
        if not session.get("adminId"):
            return redirect(url_for('admin.login'))
        error=None
        if request.method == 'POST':
            # categoryTypes = request.form.getlist('subCategory')
            data = request.form.to_dict()
            add_church = []
            category_types = []
            data_status = {"responseStatus": 0, "result": ""}
            organizationType = data['organizationType']
            id = data['editid']
            print(id)

            displayName = data['displayName']
            print("displayName")
            if request.files:
                profilePic = request.files['profileFile']
            # if 'coverFile' not in request.files:
            #     coverFile = ""
            # else:
                coverFile = request.files['coverFile']
            # profilePic = request.files['profileFile']
            # coverFile = request.files['coverFile']
            # # if profilePic.filename == '':
            #       return redirect(request.url)

            categoryId = data['category']
            # print("categoryid")
            # print(type(categoryId))
            # categoryTypes = data['subCategory']
            # print(categoryTypes)
            # email = request.form['email']
            password = data['password']
            phoneNumber = data['phoneNumber']
            websiteLink = data['websiteLink']
            streetAddress = data['streetAddress']
            countryId = data['country']
            stateId = data['state']
            cityId = data['city']
            zipcode = data['zipCode']

            if id:
                print("id")
                try:
                    queryset = PiesaGivingOrganisations.objects.get(pk__exact=id, status__in=[0,1])

                    if organizationType == 'church':
                        subCategory = data['subCategory']
                        # categoryCity = data['categoryCity']
                        # pastorType = data['pastorType']
                        treasurerType = data['treasurerType']
                        pastorProfile = request.files['pastorProfile']
                        treasurerProfile = request.files['treasurerFile']
                        pastorName = data['pastorName']
                        pastorEmail = data['pastorEmail']
                        pastorMobileNumber = data['pastorMobileNumber']
                        treasurerName = data['treasurerName']
                        treasurerEmail = data['treasurerEmail']
                        treasurerMobileNumber = data['treasurerMobileNumber']

                        if displayName and streetAddress and countryId and stateId and cityId and zipcode and categoryId and subCategory and treasurerType:
                            queryset.update(displayName=displayName, phoneNumber=phoneNumber, websiteLink=websiteLink,
                                            streetAddress=streetAddress, zipcode=zipcode, countryId=ObjectId(countryId),
                                            stateId=ObjectId(stateId), cityId=ObjectId(cityId))

                            if categoryId:
                                queryset.update(categoryId=ObjectId(categoryId))
                            # if categoryTypes:
                            #     for ctype in categoryTypes:
                            #         category_types.append(ObjectId(ctype))
                            #     # category_types.append(ObjectId(categoryTypes))
                            # queryset.update(categoryTypes=category_types)

                        else:
                            if organizationType == 'church':
                                error = "Required fields are mssing!"
                                return render_template('/super_admin/church_list.html', error=error)


                        if pastorName and treasurerName:
                            queryset.update(pastorName=pastorName,pastorMobileNumber=pastorMobileNumber,treasurerName=treasurerName,treasurerMobileNumber=treasurerMobileNumber)

                        else:
                            error = "required fields are missing!!!"
                            if organizationType == 'church':
                                return render_template('/super_admin/church_list.html', error=error)
                            elif organizationType == 'charity':
                                return render_template('/super_admin/charity_list.html', error=error)
                        # try:
                        if pastorEmail:
                            if (re.search(email_regex, pastorEmail)):
                                queryset.update(pastorEmail=pastorEmail)
                            else:
                                error = "email is not valid!!!"
                                if organizationType == 'church':
                                    return render_template('/super_admin/church_list.html', error=error)
                                elif organizationType == 'charity':
                                    return render_template('/super_admin/charity_list.html', error=error)

                        if treasurerEmail:
                            if (re.search(email_regex, treasurerEmail)):
                                queryset.update(treasurerEmail=treasurerEmail)
                            else:
                                error = "email is not valid!!!"
                                if organizationType == 'church':
                                    return render_template('/super_admin/church_list.html', error=error)
                                elif organizationType == 'charity':
                                    return render_template('/super_admin/charity_list.html', error=error)

                        # if pastorMobileNumber:
                        #     if (isValid(pastorMobileNumber)):
                        #         queryset.update(pastorMobileNumber=pastorMobileNumber)
                        #     else:
                        #         error = "phoneNumber is not valid!!!"
                        #         if organizationType == 'church':
                        #             return render_template('/super_admin/church_list.html', error=error)
                        #         elif organizationType == 'charity':
                        #             return render_template('/super_admin/charity_list.html', error=error)

                        # if treasurerMobileNumber:
                        #     if (isValid(treasurerMobileNumber)):
                        #         queryset.update(treasurerMobileNumber=treasurerMobileNumber)
                        #     else:
                        #         error = "phoneNumber is not valid!!!"
                        #         if organizationType == 'church':
                        #             return render_template('/super_admin/church_list.html', error=error)
                        #         elif organizationType == 'charity':
                        #             return render_template('/super_admin/charity_list.html', error=error)
                        #

                    elif organizationType == 'charity':
                        ceoName = request.form['ceoName']
                        ceoEmail = request.form['ceoEmail']
                        ceoMobileNumber = request.form['ceoMobileNumber']
                        cfoName = request.form['cfoName']
                        cfoEmail = request.form['cfoEmail']
                        cfoMobileNumber = request.form['cfoMobileNumber']

                        if displayName and streetAddress and countryId and stateId and cityId and zipcode and categoryId and categoryTypes:
                            queryset.update(displayName=displayName, phoneNumber=phoneNumber,
                                            websiteLink=websiteLink,
                                            streetAddress=streetAddress, zipcode=zipcode,
                                            countryId=ObjectId(countryId), stateId=ObjectId(stateId),
                                            cityId=ObjectId(cityId))

                            if categoryId:
                                queryset.update(categoryId=ObjectId(categoryId))
                            if categoryTypes:
                                for ctype in categoryTypes:
                                    category_types.append(ObjectId(ctype))
                                # category_types.append(ObjectId(categoryTypes))
                            queryset.update(categoryTypes=category_types)

                        else:
                            if organizationType == 'charity':
                                # return redirect(url_for('admin_organizations.charity_list'))
                                error = "Required fields are mssing!"
                                return render_template('/super_admin/charity_list.html', error=error)

                        if ceoName and cfoName:
                            queryset.update(ceoName=ceoName,ceoMobileNumber=ceoMobileNumber,cfoName=cfoName,cfoMobileNumber=cfoMobileNumber)

                        # if ceoMobileNumber:
                        #     if (isValid(ceoMobileNumber)):
                        #         queryset.update(ceoMobileNumber = ceoMobileNumber)
                        #     else:
                        #         error = "phoneNumber is not valid!!!"
                        #         if organizationType == 'church':
                        #             return render_template('/super_admin/church_list.html', error=error)
                        #         elif organizationType == 'charity':
                        #             return render_template('/super_admin/charity_list.html', error=error)
                        # if cfoMobileNumber:
                        #     if (isValid(cfoMobileNumber)):
                        #         queryset.update(cfoMobileNumber = cfoMobileNumber)
                        #     else:
                        #         error = "phoneNumber is not valid!!!"
                        #         if organizationType == 'church':
                        #             return render_template('/super_admin/church_list.html', error=error)
                        #         elif organizationType == 'charity':
                        #             return render_template('/super_admin/charity_list.html', error=error)

                        if ceoEmail:
                            if (re.search(email_regex, ceoEmail)):
                                queryset.update(ceoEmail=ceoEmail)
                            else:
                                error = "email is not valid!!!"
                                if organizationType == 'church':
                                    return render_template('/super_admin/church_list.html', error=error)
                                elif organizationType == 'charity':
                                    return render_template('/super_admin/charity_list.html', error=error)
                        if cfoEmail:
                                if (re.search(email_regex, cfoEmail)):
                                    queryset.update(cfoEmail=cfoEmail)
                                else:
                                    error = "email is not valid!!!"
                                    if organizationType == 'church':
                                        return render_template('/super_admin/church_list.html', error=error)
                                    elif organizationType == 'charity':
                                        return render_template('/super_admin/charity_list.html', error=error)

                    if profilePic:
                        if os.path.exists(app.config['UPLOAD_FOLDER']+"/organizations/proFile"):
                            for fname in os.listdir(app.config['UPLOAD_FOLDER']+"/organizations/proFile"):
                                if fname.startswith(str(id)+"."):
                                    os.remove(os.path.join(app.config['UPLOAD_FOLDER']+"/organizations/proFile", fname))
                        if profilePic.filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS:
                            ext = profilePic.filename.rsplit('.',1)[1].lower()
                            file_name = str(id)+"."+ext
                            if not os.path.exists(app.config['UPLOAD_FOLDER']+"/organizations/proFile"):
                                os.mkdir(app.config['UPLOAD_FOLDER']+"/organizations/proFile")
                            org_image = app.config['UPLOAD_FOLDER']+"/organizations/proFile"
                            profilePic.save(os.path.join(org_image,file_name))
                            queryset.update(profilePic = file_name)

                    if coverFile:
                        if os.path.exists(app.config['UPLOAD_FOLDER']+"/organizations"):
                            for fname in os.listdir(app.config['UPLOAD_FOLDER']+"/organizations/coverFile"):
                                if fname.startswith(str(id)+"."):
                                    os.remove(os.path.join(app.config['UPLOAD_FOLDER']+"/organizations/coverFile", fname))
                        if coverFile.filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS:
                            ext = coverFile.filename.rsplit('.',1)[1].lower()
                            file_name = str(id)+"."+ext
                            if not os.path.exists(app.config['UPLOAD_FOLDER']+"/organizations/coverFile"):
                                os.mkdir(app.config['UPLOAD_FOLDER']+"/organizations/coverFile")
                            org_image = app.config['UPLOAD_FOLDER']+"/organizations/coverFile"
                            coverFile.save(os.path.join(org_image,file_name))
                            queryset.update(coverPhoto=file_name)
                        # return redirect(url_for('admin.login'))
                    data_status["responseStatus"] = 1
                    data_status["result"] = "Organizations updated successfully!"
                    if organizationType == 'church':
                        flash("Church Record Updated Successfully")
                        return redirect(url_for('admin_organizations.church_list'))
                    elif organizationType == 'charity':
                        flash("Charity Record Updated Successfully")
                        return redirect(url_for('admin_organizations.charity_list'))
                        # return redirect(url_for('admin_organizations.charity_list'))

                    # else:
                    #     # return redirect(url_for('edit_church', id=id, organizationType=organizationType))
                    #     # data_status["responseStatus"] = 0
                    #     # data_status["result"] = "Required fields are mssing!"
                    #     if organizationType == 'church':
                    #         return render_template('/super_admin/church_list.html')
                    #         # return redirect(url_for('admin_organizations.church_list'))
                    #     elif organizationType == 'charity':
                    #         # return redirect(url_for('admin_organizations.charity_list'))
                    #         error = "Required fields are mssing!"
                    #         return render_template('/super_admin/charity_list.html', error=error)

                except Exception as e:
                    raise e
                    error = e.args[0]
                    return render_template('/super_admin/dashboard.html',error=error)


            if organizationType == 'church':
                return redirect(url_for('admin_organizations.church_list'))
            elif organizationType=='charity':
                return redirect(url_for('admin_organizations.charity_list'))


            # if displayName and profilePic and streetAddress and countryId and stateId and cityId and email and password and zipcode:
            #
            # else:
            #     error="Required Fields missing!!!"
            #     # error="Required Fields missing!!!"
            #     if organizationType == 'church':
            #         return render_template('/super_admin/add_church.html',error=error)
            #     elif organizationType == 'charity':
            #         return render_template('/super_admin/add_charity.html',error=error)

        # return render_template('/super_admin/edit_church.html')

    except Exception as err:
        app.logger.error(traceback.format_exc())
        error = "Unable to edit Organization!"
        return redirect(request.url)
        # return redirect('/edit_church/'+id+"/"+organizationType)
        # return  redirect(url_for('edit_church', id = id, organizationType=organizationType))
        # return render_template('/super_admin/dashboard.html', error = error)



